# Scraping the Judiciary Website to collect PFDs 

In [1]:
# importing libraries & setting up the Notebook  
from requests import get
from requests import ConnectionError
from bs4 import BeautifulSoup
import re
from time import sleep
from time import time
import csv
import pandas as pd

from tqdm.auto import tqdm
    
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
 
def get_url(url):
    response = get(url, verify = False)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

def retries(record_url, tries=3):
    for i in range(tries):
        try:
            soup = get_url(record_url)
            return soup
        except (ConnectionError, SSLError):
            if i < tries - 1:
                sleep(2)
                continue
            else:
                return 'Con error'

/Users/alexandercolby/opt/miniconda3/envs/preventable/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Scraper starts here - run Sun 14 May 2023 9.15 pm

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

base_url = 'https://www.judiciary.uk/prevention-of-future-death-reports/page/{}/'
# update the below number based on the number of 'pages' on the Judiciary website 
page_count = 442

with requests.Session() as session:
    record_urls = [ ]
    for page in tqdm(range(1, page_count + 1)):
        url = base_url.format(page)
        try:
            response = session.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            record_urls += [a['href'] for a in soup.find_all('a', {'class': 'card__link'})]
        except (requests.exceptions.RequestException, ValueError, AttributeError) as e:
            print(f"Failed to process page {page}: {e}")
    print(f"Collected {len(record_urls)} URLs")


100%|██████████| 442/442 [00:33<00:00, 13.12it/s]

Collected 4420 URLs


In [3]:
# Here we check how many records/cases/urls were found on the Judiciary website
len(record_urls)

4420

In [4]:
record_urls[0]

'https://www.judiciary.uk/prevention-of-future-death-reports/stephen-richardson-prevention-of-future-deaths-report/'

In [5]:
record_urls[-1]

'https://www.judiciary.uk/prevention-of-future-death-reports/rachael-dallison/'

Here is the second loop that will go through the list of URLs from above to visit each individual record to pull out and store the text data into the a database

In [6]:
def error_details(e_dict, record_count, record_url, details):
    e_dict['index'] = record_count
    e_dict['url'] = record_url
    e_dict['reason'] = details
    return e_dict

In [7]:
reg_exp = re.compile(r"’s\s|s\s|'s\s")
text_cats = ['Date of report', 'Ref', 'Deceased name', 'Coroner name', 'Coroner Area', 'Category', "This report is being sent to"]
#First, I create two lists, one for the PDFs and one for the text data
record_text = []
pdf_urls = []
ref_list = []
#I want to loop through each URL & pull out the death information and pdf link for downloading
error_catching = []
record_count = 0
record_progress = tqdm(record_urls)

for record_url in record_progress:
    try:
        error_dict = {}
        #Calling the retries function
        soup = retries(record_url, tries=5)
        
        if soup == 'Con error':
            print(f"{record_url} could not connect")
            error_catching.append(error_details(error_dict, record_count, record_url, 'Connection Error'))
            record_count +=1
            continue

        #This gets all the text fields from the website to work with
        death_info = soup.select_one('article.single__post.pfd.type-pfd').select('div > p')
        
        if not death_info:
            print(f"{record_url} produced no data")
            error_catching.append(error_details(error_dict, record_count, record_url, 'No Text Loaded'))
            record_count +=1
            continue
            
        #Our dictionary that will hold all of the text information that we will eventually append to "record_text"
        blankdict = {}
        
        #This is to handle 1 annoying record with messed up html tags
        if record_url == 'https://www.judiciary.uk/publications/roadsafety/':
            strong = death_info[0].find_all('strong')
            heads = ['date_of_report', 'ref', 'deceased_name', 'coroner_name', 'coroner_area', 'category']
            for st, h in zip(strong,heads):
                blankdict[h] = st.next_sibling.replace(':','').replace('Ref','').strip()
        #And another record with wonky html
        elif record_url == 'https://www.judiciary.uk/publications/helen-sheath/':
            brs = death_info[0].text.split('\n')
            vals = []
            for b in brs:
                vals.append(b.split(':'))
            for v in vals:
                if v[0] == "Coroners name":
                    alt = "coroner_name"
                    blankdict[alt] = v[1].strip().replace('\n','')
                elif v[0] == "Coroners Area":
                    alt = "coroner_area"
                    blankdict[alt] = v[1].strip().replace('\n','')
                else:
                    blankdict[v[0].strip().replace(' ','_').lower()] = v[1].strip().replace('\n','')
        else:        
            #looping through all of the text categories for handling
            for p in death_info:
                #This checks for blank fields and if there is nothing, it skips it
                if p.text.strip() == '':
                    pass
                #This checks for our "Normal" case in which a colon exists and the category is one of the ones we 
                #pre-specified above in the "text_cats" list
                #We also need to account here for one strange record for "Rebecca Evans" which has a weird text error
                #That we manually correct for
                elif ':' in p.text and p.text.split(':')[0] in text_cats and not 'Rebecca-EvansR.pdf' in p.text:
                    #Simply assigning the key and value from strings on either side of the colon, making everything 
                    #lower case and replacing spaces with underscores and also removing any stray semi-colons
                    text_list = p.text.split(':')
                    blankdict[text_list[0].strip().replace(' ','_').lower()] = text_list[1].strip().replace('\n','').replace('\xa0','')

                elif 'Rebecca-EvansR.pdf' in p.text:
                    #This deals with that singular odd record that currently exists as of 8 Nov 2019
                    blankdict['category'] = p.text.split(':')[1].strip().replace('\n','')
                    
                elif ':' not in p.text:
                    #If the string doesn't have a colon, we can't split on it so have to get it into dictionary format
                    #Using an alternate method that counts the length of the thing
                    if any(x in p.text for x in text_cats):
                        t = [x for x in text_cats if x in p.text][0]
                        l = len(t)
                        blankdict[t.replace(' ','_').lower()] = p.text[l+1:].replace('\n','').replace('\xa0','')
                    elif 'Coroners Area' in p.text:
                        blankdict['coroner_area'] = p.text[13:].strip().replace('\n','').replace('\xa0','')
                    else:
                        print("Something we haven't accounted for has happened")

                elif p.text.strip().count(":") == 2:
                    #This corrects for one odd record in which there are 2 colons but should generalize to fix it for
                    #any time this could happen, so long as it happens in the same way
                    text_list = p.text.split(':')
                    new_string = text_list[0] + text_list[1]
                    new_name = re.sub(reg_exp, ' ', new_string).strip()
                    blankdict[new_name.replace(' ','_').lower()] = text_list[2].strip().replace('\n','').replace('\xa0','')

                elif ':' in p.text and p.text.split(':')[0] not in text_cats:
                    #Some field names are in the form of "name_of_decesased" or "name_of_coroner" or are plural/
                    #possessive so this smashes those into our preferred naming formats
                    if 'Name of' in p.text:
                        all_text = p.text.split(':')
                        key_name = all_text[0].split(' ')
                        blankdict[key_name[2].strip() + '_name'] = all_text[-1].strip()
                    else:    
                        new_name = re.sub(reg_exp, ' ', p.text)
                        text_list = new_name.split(':')
                        blankdict[text_list[0].strip().replace(' ','_').lower()] = text_list[1].strip().replace('\n','').replace('\xa0','')
        blankdict['url'] = record_url
        
        #A small little check for duplicated ref names
        try:
            if not blankdict['ref']:
                pass
            elif blankdict['ref'] in ref_list:
                blankdict['ref'] = blankdict['ref'] + 'A'
            ref_list.append(blankdict['ref'])
        except KeyError:
            blankdict['ref'] = ''
            
        #This appends the final dict to the list
        record_text.append(blankdict)
        
        #this is a seperate process to get the PDF URLs (no matter how many there are) and adds them to their own list   
        urls = soup.select('li.related-content__item')
        pdf_list = []
        for url in urls:
            pdf_list.append(url.findNext('a').get('href'))
        pdf_urls.append(pdf_list)
        
        record_count += 1
        
    except Exception as e:
        import sys
        error_desc = f"{str(e)} occurred for {record_url} when trying to work with {p}"
        error_n = f"{record_count}: {str(e)}"
        short_desc = error_n[:50] + '...' if len(error_n) > 50 else error_n
        record_progress.set_description(short_desc)
        # print(error_desc)
        error_catching.append(error_details(error_dict, record_count, record_url, error_desc))
        
        #Saving this in case we don't like the error catching.
        #import sys
        #raise type(e)(str(e) + '\n' + 'Error for Record: {}, Field: {}'.format(record_url, p)).with_traceback(sys.exc_info()[2])

 15%|█▍        | 660/4420 [01:43<09:58,  6.28it/s]

Something we haven't accounted for has happened


 17%|█▋        | 760/4420 [01:57<08:45,  6.96it/s]

Something we haven't accounted for has happened


 18%|█▊        | 795/4420 [02:03<09:03,  6.67it/s]

...

100%|█████████▉| 4398/4420 [11:17<00:04,  4.54it/s]

https://www.judiciary.uk/prevention-of-future-death-reports/statecustodydeath/ produced no data


100%|█████████▉| 4400/4420 [11:17<00:03,  5.04it/s]

https://www.judiciary.uk/prevention-of-future-death-reports/hospitaldeath/ produced no data


100%|██████████| 4420/4420 [11:22<00:00,  6.48it/s]


Here is the third loop to save the PDFs using the deceased Ref as the file name

In [8]:
#Any errors should print out above, but you can also check the error_catching dict
#Here we just turn it into a dataframe quickly to easily view

error_df = pd.DataFrame(error_catching)
error_df

,index,url,reason
0,4391,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
1,4392,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
2,4393,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
3,4394,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
4,4395,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
5,4396,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded
6,4398,https://www.judiciary.uk/prevention-of-future-...,No Text Loaded


In [11]:
def save_file(path_string, name_string):
    with open(path_string.format(name_string), 'wb') as d:
        d.write(myfile.content)

save_path = '~/Desktop/Python/PFDs opioids/All_PDFs8/{}.pdf'

potential_names = ['ref', 'deceased_name', 'date_of_report']

record_count = 0
#This is the final scrape to actually get the URLs and change the name (when possible) to the refs
for r_t, p_u in zip(tqdm(record_text), pdf_urls):
    if not p_u:
        #If there is no pdf at all, we skip it.
        continue
    else:
        #All this does is gets the PDF and downloads it and names it after the reg
        #It looks scary and complicated but all it is doing is varying the name in the case of multiple PDFs
        #Or naming it for the deceased person if there is no Ref value
        #If there is a pdf but no ref or deceased name, this will throw an error and we can adjust.
        try:
            counter = 0
            if len(p_u) > 1:
                for p in p_u:
                    if counter == 0:
                        myfile = get(p)
                        named = False
                        for x in potential_names:
                            try:
                                if r_t[x]:
                                    save_file(save_path, r_t[x])
                                    counter +=1
                                    named = True
                                    break
                                else:
                                    continue
                            except KeyError:
                                continue
                        if not named:       
                            save_file(save_path, 'check_record_{}'.format(record_count))
                            counter +=1

                    else:
                        myfile = get(p)
                        named = False
                        for x in potential_names:
                            try:
                                if r_t[x]:
                                    save_file(save_path, r_t[x] + '_{}'.format(counter))
                                    counter +=1
                                    named = True
                                    break
                                else:
                                    continue
                            except KeyError:
                                continue
                        if not named:
                            save_file(save_path, 'check_record_{}_{}'.format(record_count, counter))
                            counter +=1
                                    
            else:
                myfile = get(p_u[0])
                named = False
                for x in potential_names:
                    try:
                        if r_t[x]:
                            save_file(save_path, r_t[x])
                            named = True
                            break
                        else:
                            continue
                    except KeyError:
                        continue
                if not named:       
                    save_file(save_path, 'check_record_{}'.format(record_count))
            
            record_count += 1
        
        except Exception as e:
            import sys
            if r_t['ref']:
                raise type(e)(str(e) + '\n' + 'Error for Record: {}'.format(r_t['ref'])).with_traceback(sys.exc_info()[2])
            else:
                raise type(e)(str(e) + '\n' + 'Error for Record Number: {}'.format(record_count)).with_traceback(sys.exc_info()[2])

  0%|          | 0/4413 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '~/Desktop/Python/PFDs opioids/All_PDFs8/2023-0209.pdf'
Error for Record: 2023-0209

This step consolidates headers that match semantically but are mispelled or similar

In [18]:
header_map = {
  "date_of_report": "date_of_report",
  "date_of_reports": "date_of_report",

  "ref": "ref",

  "deceased_name": "deceased_name",
  "deceased_names": "deceased_name",

  "coroner_name": "coroner_name",
  "coroner_names": "coroner_name",

  "coroner_area": "coroner_area",
  "coroner_areas": "coroner_area",

  "category": "category",

  "this_report_is_being_sent_to": "this_report_is_being_sent_to",
  "these_report_are_being_sent_to": "this_report_is_being_sent_to",
  "these_report_have_been_sent_to": "this_report_is_being_sent_to",
  "thi_i_being_sent_to": "this_report_is_being_sent_to",
  "thi_ha_been_sent_to": "this_report_is_being_sent_to",

  "url": "url"
}

record_text = [{header_map[k]: v for k, v in record.items()} for record in record_text]

This is my final step that puts the text data (info on the deceased/case) into a csv file & adds the date it was pulled

In [19]:
from datetime import date

headers = ['date_of_report', 'date_of_reports', 'ref', 'deceased_name', 'deceased_names', 'coroner_name', 'coroner_area', 'category', 'this_report_is_being_sent_to', 'these_report_are_being_sent_to', 'url']

with open('death_info_{}.csv'.format(date.today()), 'w', newline='', encoding='utf-8') as deaths_csv:
    writer = csv.DictWriter(deaths_csv, fieldnames=headers)
    writer.writeheader()
    for record in record_text:
        if record == {}:
            pass
        else:
            writer.writerow(record)

This is an addition few steps to check what differences there are from the June 2021 records 

In [ ]:
import os

pdfs7 = os.listdir('All_PDFs7')
pdfs8 = os.listdir('All_PDFs8')

new_not_old = set(pdfs8).difference(pdfs7)

new_not_old_list = list(new_not_old)
new_not_old_list.sort()
new_not_old_list

In [ ]:
len(new_not_old_list)

In [ ]:
feb22 = pd.read_csv('death_info_2022-02-23.csv')
jun21 = pd.read_csv('death_info_2021-06-28.csv')

In [ ]:
cols = list(jun21.columns)
merged = feb22.merge(jun21, on=cols, how='left', indicator=True)

In [ ]:
l_only = merged[merged['_merge'] == 'left_only']

In [ ]:
len(l_only)

In [ ]:
l_only

In [ ]:
l_only.to_csv(r'death_info_newfeb22.csv')

Data processing for website 

In [ ]:
feb_names = pd.read_csv('death_info_2022-02-23.csv')
feb_names.head()

In [ ]:
feb_names['deceased_name'] = feb_names['deceased_name'].fillna("")

In [ ]:
feb_names['deceased_name'] = feb_names['deceased_name'].apply(lambda x: ''.join(i[0] for i in x.split()))

In [ ]:
feb_names['deceased_name'] = feb_names['deceased_name'].str.replace('\W', '')

In [ ]:
feb_names.head()

In [ ]:
feb_names.to_csv('death_info_2022-02-23_processed.csv')